In [1]:
import pandas as pd
import numpy as np
from numpy import linalg
import matplotlib.pyplot as plt
import seaborn as sbn
from scipy.stats import invgamma
import logging

In [2]:
from notebookutils import root_dir; root_dir()

now in dir:  /Users/Jeppe/Projects/BayesFactorModel


In [3]:
from model.utils import read_clean_kv17, read_testdata1, read_party_keys, matrix, vector, party_name_from_key, generate_Tau_trace_df
from model.distributionplotter import DistributionPlotter
from model.traceplotter import TracePlotter
from model.parameterframe import ParameterFrame
from model.parameters import Parameters

In [4]:
np.random.seed(100)

In [5]:
data = read_clean_kv17(drop_party_key = True)
labels = read_party_keys()

In [6]:
data = matrix(data) #making testing different data set simple

(1215, 15)


In [7]:
sigma_array = np.array([np.random.normal()**2 for _ in range(15)])
F = np.matrix([np.random.normal(size=3) for _ in range(1215)])
Beta = np.matrix(np.random.normal(size=(15,3)))
Ystar = np.matrix(np.random.normal(size=(1215,15)))

In [8]:
tau = [- np.inf] + [-2, -0.5, 0.5, 2] + [np.inf]
Tau = dict()
for i in range(15):
    Tau[i] = tau

In [9]:
import numpy as np
from numpy import linalg
from scipy.stats import invgamma
from scipy.stats import truncnorm
import logging

from model.utils import vector


class OrderedProbitGibbsSampler():

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    def __init__(self, n_factors, data, n_answers):

        self.y = data
        self.k = n_factors
        self.p = data.shape[1]
        self.T = data.shape[0]
        self.I_k = np.identity(self.k)
        self.I_p = np.identity(self.p)
        self.n_answers = n_answers
        
        self.v = 0.5
        self.s_sq = 2
        self.C0 = 2
        self.mu0 = 1

        self.Beta_list = list()
        self.Sigma_list = list()
        self.F_list = list()
        self.Tau_list = list()
        self.Ystar_list = list()
        
        print('number of variables:', self.p, ' number of observations:', self.T)

    def f_t(self, Beta, Sigma, t):
        """Posterior of f_t"""

        y_t = self.y[[t]].T
        S_inv = linalg.inv(Sigma * self.I_p)

        scale = linalg.inv(self.I_k + np.dot(np.dot(Beta.T, S_inv), Beta))
        loc = vector(np.dot(np.dot(np.dot(scale, Beta.T), S_inv), y_t))

        return np.random.multivariate_normal(loc, scale)

    def Beta_i(self, Sigma, F, i):

        if i < self.k:

            C_i = self.C_i(F, Sigma, i)
            m_i = self.m_i(C_i, F, Sigma, i)

            B_i = np.random.multivariate_normal(m_i, C_i)
            if B_i[i] < 0:
                # print(B_i[i])  # possible bug
                #B_i = np.random.multivariate_normal(m_i, C_i)
                B_i[i] = 0

            if i < self.k:
                B_i = np.append(B_i, np.zeros(self.k - i - 1))

        elif i >= self.k:

            C_k = self.C_k(F, Sigma, i)
            m_k = self.m_k(C_k, F, Sigma, i)

            B_i = np.random.multivariate_normal(m_k, C_k)

        else:
            raise ValueError('k is {0}, i is {1} - Beta_i probs'.format(self.k, i))

        return vector(B_i)


    def C_i(self, F, Sigma, i):
        """If i <= k """

        F_i = F.T[:i + 1].T
        sigma_i = Sigma[i]
        identity_i = np.identity(i + 1)

        return linalg.inv((1 / self.C0) * identity_i + (1 / sigma_i) * np.dot(F_i.T, F_i))

    def C_k(self, F, Sigma, i):
        """if i > k"""

        sigma_i = Sigma[i]
        return linalg.inv((1 / self.C0) * self.I_k + (1 / sigma_i) * np.dot(F.T, F))

    def m_i(self, C_i, F, Sigma, i):
        """If i <= k """

        F_i = F[:, :i + 1]  # 2000 X i
        sigma_i = Sigma[i]  # 1 x 1
        ones_i = np.matrix(np.ones(i + 1)).T
        y_i = self.y[:, [i]]
        tmp = (1 / self.C0) * self.mu0 * ones_i + (1 / sigma_i) * np.dot(F_i.T, y_i)
        return vector(np.dot(C_i, tmp))

    def m_k(self, C_k, F, Sigma, i):
        """if i > k"""

        sigma_i = Sigma[i]  # 1 x 1
        ones_k = np.matrix(np.ones(self.k)).T
        y_i = self.y[:, [i]]

        tmp = (1 / self.C0) * self.mu0 * ones_k + (1 / sigma_i) * np.dot(F.T, y_i)
        return vector(np.dot(C_k, tmp))

    def calc_Beta(self):

        B = np.matrix([self.Beta_i(self.Sigma, self.F, i) for i in range(self.p)])
        self.Beta_list.append(B)
        return B

    def calc_F(self):

        F = np.matrix([self.f_t(self.Beta, self.Sigma, t) for t in range(self.T)])
        self.add('F', F)
        return F

    def calc_Ystar(self):
        
        #defined here to not make multiple call of dot product (specially for Psi)
        Y = self.y
        Psi = self.Psi
        
        Ystar = np.matrix([self.calc_Ystar_i(Psi[i], Y[i]) for i in range(self.T)])
        self.add('Ystar', Ystar)
        return Ystar
    
    def calc_Ystar_i(self, Psi_i, Y_i):
            
        Y_star_i = list()
        for question in range(self.p):
            psi_i_j = Psi_i[0,question]
            y_i_j = self.get_y_i_j(Y_i,question)
            lb, ub = self.get_bounds(y_i_j, question)
            Y_star_i.append(self.calc_Ystar_i_j(lb, ub, psi_i_j)) 
            
        return Y_star_i

    def calc_Ystar_i_j(self, lower_bound, upper_bound, mean):
        #weird implementation of python required following:
        ub = upper_bound - mean
        lb = lower_bound - mean
        return truncnorm.rvs(a = lb, b = ub, loc = mean, scale = 1)

    def get_bounds(self, y_i_j, question):
        
        tau = self.Tau[question]
        upper_bound = tau[y_i_j]
        lower_bound = tau[y_i_j - 1]
        return lower_bound, upper_bound
    
    @staticmethod
    def get_y_i_j(Y_i, question):
        """To handle unfortunate 0'es"""
        res = Y_i[0, question]
        if res == 0:
            res = 3
            
        return res
    
    def calc_Tau(self):
        
        Tau = dict()
        for question in range(self.p):
            Tau[question] = self.calc_Tau_p(question)
            
        self.add('Tau', Tau)
        return Tau
    
    def calc_Tau_p(self, p):
        """Cutpoints/answers for given quiestion p"""
        
        _tau = [self.calc_tau_l_p(p, l) for l in range(1, self.n_answers)]
        return [- np.inf] + _tau + [np.inf]
    
    def calc_tau_l_p(self, p, l):
        
        _ , Ystar_max = self.get_Ystar_min_max(p, l) 
        Ystar_min, _ = self.get_Ystar_min_max(p, l + 1)
        l_plus = self.get_tau_l_p(p, l+1)
        l_minus = self.get_tau_l_p(p, l-1)
        _min = min(Ystar_min, l_plus)
        _max = max(Ystar_max, l_minus)
        
        #print('tau', 'p=',p,', l=',l)
        #print(l_plus, Ystar_min)
        #print(l_minus, Ystar_max)
        
        try:
            return np.random.uniform(_min, _max)
        except OverflowError as e:
            print(_min, _max)
            raise e
        
    def get_Ystar_min_max(self, question, answer):
        """
        answer is the answer of question (1, 2, 3, 4, 5)"""
        Y_l = self.y[:,question]
        Ystar_l = self.Ystar[:,question]
        
        #Ystar observation where y_l == 0
        Ystar_legal = []
 
        for i in range(self.T):
            if Y_l[i] == answer :
                Ystar_legal.append(Ystar_l[i])
        return float(min(Ystar_legal)), float(max(Ystar_legal))
    
    def get_tau_l_p(self, p, l):
        return self.Tau[p][l]
        
    @property
    def Beta(self):
        return self.Beta_list[-1]

    @property
    def F(self):
        return self.F_list[-1]

    @property
    def Sigma(self):
        #return self.Sigma_list[-1]
        return [1 for _ in range(self.p)]

    @property
    def Psi(self):
        return np.dot(self.F ,self.Beta.T)
    
    @property
    def Tau(self):
        return self.Tau_list[-1]

    @property
    def Ystar(self):
        return self.Ystar_list[-1]
    
    def add(self, param, value):
        """ add to Sigma_list, Beta_list or F_list

        Parameters
        ==========
        param: (str)
            string that should be of {'Sigma', 'F', 'Beta'}
        value: (obj)
            appropriate object for given list

        """

        if param == 'Sigma':
            self.Sigma_list.append(value)
            
        elif param == 'Tau':
            self.Tau_list.append(value)
            
        elif param == 'Ystar':
            self.Ystar_list.append(value)

        elif param == 'F':
            self.F_list.append(value)

        elif param == 'Beta':
            self.Beta_list.append(value)

        else:
            raise ValueError("Param must be in {'F', 'Sigma', 'Beta', 'Ystar', 'Tau'}")

    def sampler(self, n_iterations):

        logging.info("Sampling begins")
        for i in range(n_iterations):

            self.calc_Ystar()
            self.calc_Tau()
            self.calc_F()
            self.calc_Beta()
            if (i % 5 == 0):
                logging.info("run {0} simulations".format(i))


In [10]:
gs = OrderedProbitGibbsSampler(3, data, n_answers = 5)
gs.add('Sigma',sigma_array)
gs.add('F', F)
gs.add('Beta',Beta)
gs.add('Tau', Tau)

number of variables: 15  number of observations: 1215


In [11]:
gs.sampler(10000)

2018-11-17 12:00:40,636 : INFO : Sampling begins
2018-11-17 12:00:45,653 : INFO : run 0 simulations
2018-11-17 12:01:06,122 : INFO : run 5 simulations
2018-11-17 12:01:28,613 : INFO : run 10 simulations
2018-11-17 12:01:50,523 : INFO : run 15 simulations
2018-11-17 12:02:09,976 : INFO : run 20 simulations
2018-11-17 12:02:29,082 : INFO : run 25 simulations
2018-11-17 12:02:48,275 : INFO : run 30 simulations
2018-11-17 12:03:09,366 : INFO : run 35 simulations
2018-11-17 12:03:28,176 : INFO : run 40 simulations
2018-11-17 12:03:49,345 : INFO : run 45 simulations
2018-11-17 12:04:07,300 : INFO : run 50 simulations
2018-11-17 12:04:25,329 : INFO : run 55 simulations
2018-11-17 12:04:43,571 : INFO : run 60 simulations
2018-11-17 12:05:01,910 : INFO : run 65 simulations
2018-11-17 12:05:20,112 : INFO : run 70 simulations
2018-11-17 12:05:38,000 : INFO : run 75 simulations
2018-11-17 12:05:56,007 : INFO : run 80 simulations
2018-11-17 12:06:14,379 : INFO : run 85 simulations
2018-11-17 12:06:

2018-11-17 12:54:57,307 : INFO : run 775 simulations
2018-11-17 12:55:15,516 : INFO : run 780 simulations
2018-11-17 12:55:33,472 : INFO : run 785 simulations
2018-11-17 12:55:51,498 : INFO : run 790 simulations
2018-11-17 12:56:09,436 : INFO : run 795 simulations
2018-11-17 12:56:27,627 : INFO : run 800 simulations
2018-11-17 12:56:45,670 : INFO : run 805 simulations
2018-11-17 12:57:03,806 : INFO : run 810 simulations
2018-11-17 12:57:21,840 : INFO : run 815 simulations
2018-11-17 12:57:39,996 : INFO : run 820 simulations
2018-11-17 12:57:58,113 : INFO : run 825 simulations
2018-11-17 12:58:16,095 : INFO : run 830 simulations
2018-11-17 12:58:34,225 : INFO : run 835 simulations
2018-11-17 12:58:52,361 : INFO : run 840 simulations
2018-11-17 12:59:10,511 : INFO : run 845 simulations
2018-11-17 12:59:28,676 : INFO : run 850 simulations
2018-11-17 12:59:46,627 : INFO : run 855 simulations
2018-11-17 13:00:04,646 : INFO : run 860 simulations
2018-11-17 13:00:22,760 : INFO : run 865 simul

2018-11-17 13:40:56,644 : INFO : run 1540 simulations
2018-11-17 13:41:14,525 : INFO : run 1545 simulations
2018-11-17 13:41:32,797 : INFO : run 1550 simulations
2018-11-17 13:41:50,729 : INFO : run 1555 simulations
2018-11-17 13:42:09,096 : INFO : run 1560 simulations
2018-11-17 13:42:27,016 : INFO : run 1565 simulations
2018-11-17 13:42:45,134 : INFO : run 1570 simulations
2018-11-17 13:43:03,244 : INFO : run 1575 simulations
2018-11-17 13:43:21,354 : INFO : run 1580 simulations
2018-11-17 13:43:39,473 : INFO : run 1585 simulations
2018-11-17 13:43:57,443 : INFO : run 1590 simulations
2018-11-17 13:44:15,286 : INFO : run 1595 simulations
2018-11-17 13:44:33,595 : INFO : run 1600 simulations
2018-11-17 13:44:51,526 : INFO : run 1605 simulations
2018-11-17 13:45:09,339 : INFO : run 1610 simulations
2018-11-17 13:45:27,545 : INFO : run 1615 simulations
2018-11-17 13:45:45,504 : INFO : run 1620 simulations
2018-11-17 13:46:03,587 : INFO : run 1625 simulations
2018-11-17 13:46:21,646 : IN

2018-11-17 14:26:38,187 : INFO : run 2300 simulations
2018-11-17 14:26:56,189 : INFO : run 2305 simulations
2018-11-17 14:27:14,380 : INFO : run 2310 simulations
2018-11-17 14:27:32,411 : INFO : run 2315 simulations
2018-11-17 14:27:50,438 : INFO : run 2320 simulations
2018-11-17 14:28:08,543 : INFO : run 2325 simulations
2018-11-17 14:28:26,562 : INFO : run 2330 simulations
2018-11-17 14:28:44,464 : INFO : run 2335 simulations
2018-11-17 14:29:02,393 : INFO : run 2340 simulations
2018-11-17 14:29:20,407 : INFO : run 2345 simulations
2018-11-17 14:29:38,316 : INFO : run 2350 simulations
2018-11-17 14:29:56,266 : INFO : run 2355 simulations
2018-11-17 14:30:14,346 : INFO : run 2360 simulations
2018-11-17 14:30:32,292 : INFO : run 2365 simulations
2018-11-17 14:30:50,219 : INFO : run 2370 simulations
2018-11-17 14:31:08,026 : INFO : run 2375 simulations
2018-11-17 14:31:26,119 : INFO : run 2380 simulations
2018-11-17 14:31:44,091 : INFO : run 2385 simulations
2018-11-17 14:32:02,629 : IN

2018-11-17 15:12:23,383 : INFO : run 3060 simulations
2018-11-17 15:12:41,406 : INFO : run 3065 simulations
2018-11-17 15:12:59,532 : INFO : run 3070 simulations
2018-11-17 15:13:17,568 : INFO : run 3075 simulations
2018-11-17 15:13:35,456 : INFO : run 3080 simulations
2018-11-17 15:13:53,189 : INFO : run 3085 simulations
2018-11-17 15:14:10,996 : INFO : run 3090 simulations
2018-11-17 15:14:28,728 : INFO : run 3095 simulations
2018-11-17 15:14:46,756 : INFO : run 3100 simulations
2018-11-17 15:15:04,718 : INFO : run 3105 simulations
2018-11-17 15:15:22,716 : INFO : run 3110 simulations
2018-11-17 15:15:40,683 : INFO : run 3115 simulations
2018-11-17 15:15:58,863 : INFO : run 3120 simulations
2018-11-17 15:16:16,805 : INFO : run 3125 simulations
2018-11-17 15:16:34,611 : INFO : run 3130 simulations
2018-11-17 15:16:52,872 : INFO : run 3135 simulations
2018-11-17 15:17:10,732 : INFO : run 3140 simulations
2018-11-17 15:17:29,079 : INFO : run 3145 simulations
2018-11-17 15:17:47,028 : IN

...

2018-11-17 19:01:29,880 : INFO : run 6860 simulations
2018-11-17 19:01:48,009 : INFO : run 6865 simulations
2018-11-17 19:02:06,393 : INFO : run 6870 simulations
2018-11-17 19:02:24,451 : INFO : run 6875 simulations
2018-11-17 19:02:42,628 : INFO : run 6880 simulations
2018-11-17 19:03:00,793 : INFO : run 6885 simulations
2018-11-17 19:03:19,003 : INFO : run 6890 simulations
2018-11-17 19:03:36,983 : INFO : run 6895 simulations
2018-11-17 19:03:54,998 : INFO : run 6900 simulations
2018-11-17 19:04:12,836 : INFO : run 6905 simulations
2018-11-17 19:04:30,905 : INFO : run 6910 simulations
2018-11-17 19:04:48,917 : INFO : run 6915 simulations
2018-11-17 19:05:06,939 : INFO : run 6920 simulations
2018-11-17 19:05:25,142 : INFO : run 6925 simulations
2018-11-17 19:05:43,192 : INFO : run 6930 simulations
2018-11-17 19:06:01,105 : INFO : run 6935 simulations
2018-11-17 19:06:19,224 : INFO : run 6940 simulations
2018-11-17 19:06:37,114 : INFO : run 6945 simulations
2018-11-17 19:06:55,146 : IN

2018-11-17 19:47:22,244 : INFO : run 7620 simulations
2018-11-17 19:47:40,129 : INFO : run 7625 simulations
2018-11-17 19:47:58,113 : INFO : run 7630 simulations
2018-11-17 19:48:16,016 : INFO : run 7635 simulations
2018-11-17 19:48:34,144 : INFO : run 7640 simulations
2018-11-17 19:48:52,031 : INFO : run 7645 simulations
2018-11-17 19:49:09,922 : INFO : run 7650 simulations
2018-11-17 19:49:27,815 : INFO : run 7655 simulations
2018-11-17 19:49:45,755 : INFO : run 7660 simulations
2018-11-17 19:50:03,720 : INFO : run 7665 simulations
2018-11-17 19:50:21,706 : INFO : run 7670 simulations
2018-11-17 19:50:39,790 : INFO : run 7675 simulations
2018-11-17 19:50:57,736 : INFO : run 7680 simulations
2018-11-17 19:51:15,810 : INFO : run 7685 simulations
2018-11-17 19:51:33,715 : INFO : run 7690 simulations
2018-11-17 19:51:51,805 : INFO : run 7695 simulations
2018-11-17 19:52:09,806 : INFO : run 7700 simulations
2018-11-17 19:52:27,626 : INFO : run 7705 simulations
2018-11-17 19:52:45,633 : IN

2018-11-17 20:33:05,386 : INFO : run 8380 simulations
2018-11-17 20:33:23,554 : INFO : run 8385 simulations
2018-11-17 20:33:41,508 : INFO : run 8390 simulations
2018-11-17 20:33:59,351 : INFO : run 8395 simulations
2018-11-17 20:34:17,219 : INFO : run 8400 simulations
2018-11-17 20:34:35,339 : INFO : run 8405 simulations
2018-11-17 20:34:53,411 : INFO : run 8410 simulations
2018-11-17 20:35:11,380 : INFO : run 8415 simulations
2018-11-17 20:35:29,513 : INFO : run 8420 simulations
2018-11-17 20:35:47,543 : INFO : run 8425 simulations
2018-11-17 20:36:05,746 : INFO : run 8430 simulations
2018-11-17 20:36:23,704 : INFO : run 8435 simulations
2018-11-17 20:36:41,802 : INFO : run 8440 simulations
2018-11-17 20:36:59,657 : INFO : run 8445 simulations
2018-11-17 20:37:17,780 : INFO : run 8450 simulations
2018-11-17 20:37:36,237 : INFO : run 8455 simulations
2018-11-17 20:37:54,499 : INFO : run 8460 simulations
2018-11-17 20:38:12,328 : INFO : run 8465 simulations
2018-11-17 20:38:30,460 : IN

2018-11-17 21:18:52,837 : INFO : run 9140 simulations
2018-11-17 21:19:10,877 : INFO : run 9145 simulations
2018-11-17 21:19:29,063 : INFO : run 9150 simulations
2018-11-17 21:19:47,200 : INFO : run 9155 simulations
2018-11-17 21:20:05,167 : INFO : run 9160 simulations
2018-11-17 21:20:23,249 : INFO : run 9165 simulations
2018-11-17 21:20:41,145 : INFO : run 9170 simulations
2018-11-17 21:20:59,025 : INFO : run 9175 simulations
2018-11-17 21:21:16,912 : INFO : run 9180 simulations
2018-11-17 21:21:35,222 : INFO : run 9185 simulations
2018-11-17 21:21:53,341 : INFO : run 9190 simulations
2018-11-17 21:22:11,143 : INFO : run 9195 simulations
2018-11-17 21:22:29,100 : INFO : run 9200 simulations
2018-11-17 21:22:47,144 : INFO : run 9205 simulations
2018-11-17 21:23:05,220 : INFO : run 9210 simulations
2018-11-17 21:23:23,158 : INFO : run 9215 simulations
2018-11-17 21:23:41,350 : INFO : run 9220 simulations
2018-11-17 21:23:59,462 : INFO : run 9225 simulations
2018-11-17 21:24:17,669 : IN

2018-11-17 22:04:37,946 : INFO : run 9900 simulations
2018-11-17 22:04:56,078 : INFO : run 9905 simulations
2018-11-17 22:05:14,462 : INFO : run 9910 simulations
2018-11-17 22:05:32,559 : INFO : run 9915 simulations
2018-11-17 22:05:50,786 : INFO : run 9920 simulations
2018-11-17 22:06:09,161 : INFO : run 9925 simulations
2018-11-17 22:06:27,463 : INFO : run 9930 simulations
2018-11-17 22:06:45,567 : INFO : run 9935 simulations
2018-11-17 22:07:03,414 : INFO : run 9940 simulations
2018-11-17 22:07:21,453 : INFO : run 9945 simulations
2018-11-17 22:07:39,346 : INFO : run 9950 simulations
2018-11-17 22:07:57,380 : INFO : run 9955 simulations
2018-11-17 22:08:15,537 : INFO : run 9960 simulations
2018-11-17 22:08:33,654 : INFO : run 9965 simulations
2018-11-17 22:08:51,635 : INFO : run 9970 simulations
2018-11-17 22:09:09,555 : INFO : run 9975 simulations
2018-11-17 22:09:27,862 : INFO : run 9980 simulations
2018-11-17 22:09:45,938 : INFO : run 9985 simulations
2018-11-17 22:10:03,988 : IN

In [12]:
beta_params = ParameterFrame(gs.Beta_list, 'beta')
beta_trace_estimation = beta_params.get_trace_df()
beta_trace_estimation.to_pickle('data//probit_estimation_beta_trace_df.pkl')

In [13]:
factor = ParameterFrame(gs.F_list, 'factor_estimation')
factor_trace_estimation = factor.get_trace_df()
factor_trace_estimation.to_pickle('data//probit_estimation_factor_trace_df.pkl')

In [14]:
ystar = ParameterFrame(gs.Ystar_list, 'ystar')
ystar_trace_estimation = ystar.get_trace_df()
ystar_trace_estimation.to_pickle('data//probit_ystar_trace_df.pkl')

In [15]:
tau_trace = generate_Tau_trace_df(gs.Tau_list)
tau_trace.to_pickle('data//probit_tau_trace_df.pkl')